# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*

#Question 1
          
"""
We can use "Holdout" in order to evaluate how the model performs on data it hasn't see during training. We then devide available data into a training set, a validation set and a test set. We train different models on training test, chose model that gives the best results on validation set and we then evaluate model's performance on unseen data thanks to the test set.  

However a second strategy is most commonly used in most deep learning applications as data is not unlimited. In that case, we can use cross-validation and evaluate models on different test sets coming from one single data set.  
        
"""

#Question 2

"""
If it feels like our Multi-layer fully connected Neural Network is overfitting we can do those four things to improve its performance :

- Add a regularization term (L1 or L2) in order to encourage smaller values for the parameters
- Cross validation
- Data augmentation 
- Dropout / Dropconnect
    
"""


## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*


#Question 1:

""" 
Sigmoid or hyperbolic tangent fonction have small gradient values for large input values. In that case, backpropagation won't be able to change weights value enougth and our model won't be able to learn. This is a problem called "vanishing gradient".
"""
#Question 2: 

"""
If we use sigmoid as the output layer's activation function, we will only compute one single sigmoid's gradient during backpropagation. Therefore, it is then less likely to be subject to vanishing gradient and then to poor learning results.
"""
#Question 3: 

"""
If we set all weights to zero in a multi-layer fully connected neural network and attempt to train the model with GD, we will have the same propagation of the gradient on each neuron of each layer? Therefore, the same thing would be added to each weight and each neuron will learn the same way. We then basically have a logistic regression model. 

If we do the same for a logistic regression model we won't have any trouble, we only have a "single neuron" to train.
"""



## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *
import numpy as np
import matplotlib.pyplot as plt

In [41]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd
        self.batch_size = 10

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        m = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([weight_init_sd]))
        self.params = m.sample((10,784)).view(10,784) #parameters with shape (batch_size,784)
        self.params.requires_grad_()
        
        self.bias = m.sample((10,1)).view(10) #bias
        self.bias.requires_grad_()

        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        b_size = inputs.size(0) #obtain batch size
        b_input = inputs.view(b_size,-1) #flatten inputs to (batch_size, 784)
        output = torch.Tensor(b_size,10) #initialize output 
        
         #stability element
        
        for i in range(b_size) :
            
            prod = self.params@b_input[i] + self.bias# y = Wx + b
            
            b = prod.max() #adding element for log_softmax computation
            b = b.item()
            
            stab = torch.ones(10)*b
            
            prod_exp = (prod - stab).exp()
            stab_exp = stab.exp()
            
            p = prod_exp*stab_exp
            
            S = torch.ones(10)*p.sum()
            
            Softmax = prod-stab - S.log()
            
            output[i] = Softmax
            
        return(output)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return([self.params,self.bias])
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        pos_params = self.params.abs()
        
        return(torch.sum(pos_params))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        sqrd_params = self.params*self.params
        
        return(torch.sum(sqrd_params))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [45]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
         
model = MultinomialLogisticRegressionClassifier(weight_init_sd=0.8)
res = run_experiment(
    model,
    optimizer=optim.Adamax(model.parameters(), 0.01),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.0,
    l2_penalty_coef=0.0001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 14.0584, Accuracy: 0.7296
Validation set:	Average loss: 14.2841, Accuracy: 0.8493

Epoch 1: training...
Train set:	Average loss: 14.2917, Accuracy: 0.8666
Validation set:	Average loss: 14.1009, Accuracy: 0.8857

Epoch 2: training...
Train set:	Average loss: 13.7199, Accuracy: 0.8895
Validation set:	Average loss: 13.3455, Accuracy: 0.8980

Epoch 3: training...
Train set:	Average loss: 12.9650, Accuracy: 0.9003
Validation set:	Average loss: 12.5657, Accuracy: 0.9017

Epoch 4: training...
Train set:	Average loss: 12.3030, Accuracy: 0.9071
Validation set:	Average loss: 11.8887, Accuracy: 0.9103

Epoch 5: training...
Train set:	Average loss: 11.7493, Accuracy: 0.9116
Validation set:	Average loss: 11.5511, Accuracy: 0.9127

Epoch 6: training...
Train set:	Average loss: 11.2987, Accuracy: 0.9151
Validation set:	Average loss: 11.0302, Accuracy: 0.9170

Epoch 7: training...
Train set:	Average loss: 10.8422, Accuracy: 0.9184
Validation set:	Average l

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [6]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()
        
        m = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([weight_init_sd]))
        
        self.dense1 = m.sample((128,784)).view(128,784) #parameters with shape (batch_size,784) 
        self.dense1.requires_grad_()
        
        self.b1 = m.sample((128, 1)).view(128,)
        self.b1.requires_grad_()
        
        self.dense2 = m.sample((64,128)).view(64,128)
        self.dense2.requires_grad_()
        
        self.b2 = m.sample((64, 1)).view(64,)
        self.b2.requires_grad_()
        
        self.dense3 = m.sample((32,64)).view(32,64)
        self.dense3.requires_grad_()
        
        self.b3 = m.sample((32, 1)).view(32,)
        self.b3.requires_grad_()
        
        self.dense4 = m.sample((10,32)).view(10,32)
        self.dense4.requires_grad_()
        
        self.b4 = m.sample((10, 1)).view(10,)
        self.b4.requires_grad_()
        
        self.layers = [self.dense1,self.b1,self.dense2,self.b2,self.dense3,self.b3,self.dense4,self.b4]
        

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    
    #def plot_results(self,)
    
    
    def activ(self, y): #log_softmax
        
        y_exp = y.exp()
            
        S = torch.ones(y.size(0))*y_exp.sum()
            
        return(y - S.log())
        

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        b_size = inputs.size(0) #obtain batch size
        b_input = inputs.view(b_size,-1) #flatten inputs to (batch_size, 784)
        
        output = torch.Tensor(b_size,10) #initialize output 
        
        layer1_out = torch.zeros(128)
        layer2_out = torch.zeros(64)
        layer3_out = torch.zeros(32)

        for i in range(0,b_size) :
            
            layer1_out = self.dense1@b_input[i] + self.b1 # 128 x 1
            layer1_out = self.activation(layer1_out)

            layer2_out = self.dense2@layer1_out + self.b2 # 64 x 1
            layer2_out = self.activation(layer2_out)
            
            layer3_out = self.dense3@layer2_out + self.b3 # 32 x 1
            layer3_out = self.activation(layer3_out)

            output[i] = self.dense4@layer3_out + self.b4 # 10 x 1
            output[i] = self.activ(output[i])
            
        return(output)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return([self.dense1,self.b1,self.dense2,self.b2,self.dense3,self.b3,self.dense4,self.b4])
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        n = len(self.layers)
        L1norm = np.zeros(n)
        
        for i in range(0,n):
            
            lay_abs = self.layers[i].abs()
            L1norm[i] = lay_abs.sum()
            
        return(sum(L1norm))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        n = len(self.layers)
        L2norm = np.zeros(n)
        
        for i in range(0,n):
            
            lay_abs = self.layers[i] * self.layers[i]
            L2norm[i] = lay_abs.sum()
            
        return(sum(L2norm))
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [12]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun="relu",weight_init_sd=0.2)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), 0.001),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.01,
    l2_penalty_coef=0.01,
    suppress_output=False
)



Epoch 0: training...
Train set:	Average loss: 0.4102, Accuracy: 0.8752
Validation set:	Average loss: 0.1916, Accuracy: 0.9427

Epoch 1: training...
Train set:	Average loss: 0.1484, Accuracy: 0.9557
Validation set:	Average loss: 0.1479, Accuracy: 0.9567

Epoch 2: training...
Train set:	Average loss: 0.1015, Accuracy: 0.9687
Validation set:	Average loss: 0.1314, Accuracy: 0.9625

Epoch 3: training...
Train set:	Average loss: 0.0752, Accuracy: 0.9772
Validation set:	Average loss: 0.1228, Accuracy: 0.9667

Epoch 4: training...
Train set:	Average loss: 0.0588, Accuracy: 0.9814
Validation set:	Average loss: 0.1079, Accuracy: 0.9688

Epoch 5: training...
Train set:	Average loss: 0.0462, Accuracy: 0.9850
Validation set:	Average loss: 0.1140, Accuracy: 0.9687

Epoch 6: training...
Train set:	Average loss: 0.0347, Accuracy: 0.9891
Validation set:	Average loss: 0.1008, Accuracy: 0.9708

Epoch 7: training...
Train set:	Average loss: 0.0292, Accuracy: 0.9904
Validation set:	Average loss: 0.1250, Ac

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*

"""
3. Alternative strategy for initializing weights :

In order to prevent from vanishing gradient issue we can sample weights from normal distribution with specific variances depending on the activation function we use. 
For example, if we use Relu, we can multiply sample by sqrt(2/data_size).
"""